In [1]:
%matplotlib inline

import os
import json
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm as loadingbar

In [2]:
from unified_image_reader import Image
from dataset import Dataset, LabelManager, label_extractor
from filtration import FilterBlackAndWhite, FilterHSV, FilterFocusMeasure, FilterManager

In [3]:
import cell_level

In [4]:
features = cell_level.feature_engineering.FeatureManager(as_vector=True)
feature_dataframe = pd.DataFrame(columns = ['RegionNumFiltered']+list(features.get_empty_feature_set().keys()))
feature_dataframe

,RegionNumFiltered,MeanHSV0,MeanHSV1,MeanHSV2,HueHistogram0,HueHistogram1,HueHistogram2,HueHistogram3,HueHistogram4,HueHistogram5,...,Perimeter0,AreaPerimeter0,InsideRadialContact0,InsideRadialContact1,InsideRadialContact2,InsideRadialContact3,InsideRadialContact4,InsideRadialContact5,InsideRadialContact6,InsideRadialContact7


In [ ]:
model_manager = cell_level.segmentation.ModelManager()
model = model_manager.load_model("StarDist")

In [5]:
imgs_path = "/workspaces/dev-container/testing/training-simple/test-wsi/0/"
dataset = Dataset(
    data_dir = imgs_path,
    labels = LabelManager(imgs_path, label_extractor.LabelExtractorNoLabels()),
    filtration = FilterManager([
        FilterBlackAndWhite(),
        FilterHSV(),
        FilterFocusMeasure()
    ])
)

preprocessing file /workspaces/dev-container/testing/training-simple/test-wsi/0/84429T_003.tif


In [ ]:
for region_num in loadingbar(range(len(dataset)), total=len(dataset)):
    region, _ = dataset[region_num]
    cells = model.get_instances(region)
    if len(cells) == 0:
        continue
    for cell_num, cell_instance in loadingbar(enumerate(cells), leave=False):
        feature_data = features(cell_instance)
        feature_data['RegionNumFiltered'] = region_num
        feature_dataframe = feature_dataframe.append(
            feature_data,
            ignore_index = True
        )
feature_dataframe.to_csv('features.csv')

In [7]:
group = dataset.filtration_cache.h5file.root._f_get_child('["<FilterBlackAndWhite:{\'filter_threshold\':0.5,\'binarization_threshold\':0.85,\'rgb_weights\':[0.2989,0.587,0.114]}>","<FilterHSV:{\'threshold\':100}>","<FilterFocusMeasure:{\'threshold\':65.0}>"]')

In [8]:
table = group._f_get_child('84429T_003_DOTSYMBOL_tif')
table

/["<FilterBlackAndWhite:{'filter_threshold':0.5,'binarization_threshold':0.85,'rgb_weights':[0.2989,0.587,0.114]}>","<FilterHSV:{'threshold':100}>","<FilterFocusMeasure:{'threshold':65.0}>"]/84429T_003_DOTSYMBOL_tif (Table(16625,)) '84429T_003_DOTSYMBOL_tif'
  description := {
  "region_index_base": Int32Col(shape=(), dflt=0, pos=0),
  "region_index_target": Int32Col(shape=(), dflt=0, pos=1),
  "filtration_status": BoolCol(shape=(), dflt=False, pos=2)}
  byteorder := 'little'
  chunkshape := (7281,)

In [9]:
True in table.col("filtration_status")

False

In [17]:
from unified_image_reader import Image
from filtration import FilterBlackAndWhite, FilterHSV, FilterFocusMeasure, FilterManager
from tqdm import tqdm as loadingbar

filtermanager = FilterManager([
    FilterBlackAndWhite(),
    FilterHSV(),
    FilterFocusMeasure()
])
filterblackandwhite = FilterBlackAndWhite()
filterhsv = FilterHSV()
filterfocusmeasure = FilterFocusMeasure()

all_filters = [filtermanager, filterblackandwhite, filterhsv, filterfocusmeasure]
statuses = {type(f).__name__: (lambda : {True: 0, False: 0})() for f in all_filters}
print(statuses)

img_path = "/workspaces/dev-container/testing/training-simple/test-wsi/0/84429T_003.tif"
for i, region in loadingbar(enumerate(Image(img_path)), total = Image(img_path).number_of_regions()):
    for f in all_filters:
        filter_specific_status_tracker = statuses[type(f).__name__]
        status = f(region)
        if status is None: raise Exception(i, type(f).__name__)
        filter_specific_status_tracker[f(region)] += 1

print(statuses)

{'FilterManager': {True: 0, False: 0}, 'FilterBlackAndWhite': {True: 0, False: 0}, 'FilterHSV': {True: 0, False: 0}, 'FilterFocusMeasure': {True: 0, False: 0}}


  0%|          | 2/16625 [00:00<14:57, 18.52it/s]

None
0 FilterManager
None
1 FilterManager
None


  0%|          | 5/16625 [00:00<11:19, 24.46it/s]

2 FilterManager
None
3 FilterManager
None
4 FilterManager
None
5 FilterManager


...

  1%|          | 104/16625 [00:03<09:33, 28.82it/s]

101 FilterManager
None
102 FilterManager
None
103 FilterManager


  1%|          | 104/16625 [00:04<10:36, 25.97it/s]


None


KeyboardInterrupt: 

In [ ]:
type(FilterFocusMeasure()).__name__

In [ ]:
plt.imshow(region)
print(region.shape)
print(region_num)
print(cell_num)
print(cell_instance.image.shape, cell_instance.mask.shape)
print(cell_instance.data['score'])

In [ ]:
plt.imshow(cell_instance.image)

In [ ]:
plt.imshow(cell_instance.mask)

In [ ]:
feature_dataframe